In [1]:
import numpy as np
import pandas as pd
import xpress as xp

# Create a problem called EV charging station

prob = xp.problem(name='EV charging station')

# Defining the index sets

number_of_types = 3
number_of_grids = 435
number_of_years = 4
number_of_ppmax = 18 # The max number of potential charging points in one grid
number_of_poimax = 28 # The max number of interest points in one grid
number_of_cpmax = 10

types = range(number_of_types)
grids = range(number_of_grids)
years = range(number_of_years)
ppmax = range(number_of_ppmax)
poimax = range(number_of_poimax)
cpmax = range(number_of_cpmax)



In [2]:
demand_data = pd.read_excel('Demand_data.xlsx')
demand = demand_data[['Demand_0', 'Demand_1', 'Demand_2', 'Demand_3']].to_numpy()
demand = np.insert(demand, 0, [0,0,0,0], axis=0) # Let the index i denote the i-th grid


In [3]:
# import re
# nb = demand_data['NEIGHBORS']
# neighbor = np.zeros((number_of_grids+1,8))
# for index in grids:
#     nb_split = re.findall("\d+", nb[index])
#     neighbor[index+1, 0:len(nb_split)] = np.array(nb_split)

# neighbor_demand = np.zeros((number_of_grids+1, number_of_years))
# for index in range(1, number_of_grids+1):
#     for n in neighbor[index]:
#         if n == 0:
#             break
#         for t in years:    
#             neighbor_demand[index, t] += demand[int(n),t]
# neighbor_demand[398,2]


475.5244925393377

In [3]:
loc_charging = pd.read_excel('Charging_points_00.xlsx')
loc_charging = loc_charging[['Latitude', 'Longitude','grid number','Slow','Fast','Rapid']].sort_values('grid number').reset_index()
#len(loc_charging['Latitude'].unique()) == len(loc_charging['index'])

In [4]:
cp_len = len(loc_charging.index)

# Collect latitude of each charging point
cp_lati = np.zeros((number_of_grids,number_of_cpmax)) 
for index in range(cp_len): 
    j = int(loc_charging.iloc[index]['grid number']) 
    for p in cpmax: 
        if cp_lati[j,p] == 0: 
            cp_lati[j,p] = loc_charging.iloc[index]['Latitude'] 
            break
            
# Collect longitude of each charging point
cp_longi = np.zeros((number_of_grids,number_of_cpmax)) 
for index in range(cp_len): 
    j = int(loc_charging.iloc[index]['grid number']) 
    for p in cpmax: 
        if cp_longi[j,p] == 0: 
            cp_longi[j,p] = loc_charging.iloc[index]['Longitude'] 
            break

# If grid i has p charging points now, then cp_0[j, 0:p] will be 1
# and 0 otherwise
cp_0 = np.zeros((number_of_grids,number_of_cpmax)) 
for index in range(cp_len): 
    j = int(loc_charging.iloc[index]['grid number']) 
    for p in cpmax: 
        if cp_0[j,p] == 0: 
            cp_0[j,p] = 1 
            break

cp_0_copy = np.copy(cp_0)
# Collect the number of different types of chargers in each existing station
charger_0 = np.zeros((number_of_types,number_of_grids,number_of_cpmax)) 
for index in range(cp_len): 
    j = int(loc_charging.iloc[index]['grid number']) 
    for p in cpmax: 
        if cp_0_copy[j,p] == 1: 
            charger_0[0,j,p] = loc_charging.iloc[index]['Slow'] 
            charger_0[1,j,p] = loc_charging.iloc[index]['Fast'] 
            charger_0[2,j,p] = loc_charging.iloc[index]['Rapid'] 
            cp_0_copy[j,p] = 0 
            break


In [5]:
poi = pd.read_excel('Interest _points.xlsx')
poi = poi[['Latitude', 'Longitude', 'grid number']].sort_values('grid number').reset_index()
#len(poi['Latitude'].unique()) == len(poi['index'])

In [6]:
poi_len = len(poi.index)

# Collect latitude of each interest point
poi_lati = np.zeros((number_of_grids,number_of_poimax))
for index in range(poi_len):
    j = int(poi.iloc[index]['grid number'])
    for g in poimax:
        if poi_lati[j,g] == 0:
            poi_lati[j,g] = poi.iloc[index]['Latitude']
            break
            
# Collect longitude of each interest point
poi_longi = np.zeros((number_of_grids,number_of_poimax))
for index in range(poi_len):
    j = int(poi.iloc[index]['grid number'])
    for g in poimax:
        if poi_longi[j,g] == 0:
            poi_longi[j,g] = poi.iloc[index]['Longitude']
            break
            
    

In [7]:
potential_loc = pd.read_excel('Potential_charging_points.xlsx')
pcp = potential_loc[['Latitude', 'Longitude','grid number']].sort_values('grid number').reset_index()
pd.set_option('display.max_rows', None)

# print(sum(pcp['Latitude'].isin(loc_charging['Latitude'])))
# print(sum(loc_charging['Latitude'].isin(pcp['Latitude'])))
# print(loc_charging[loc_charging['grid number'].isin(pcp['grid number'])].reset_index())

In [8]:
pcp_len = len(pcp.index)

# pcp_bi[j,0:k] = 1 if the grid j has k potential points
pcp_bi = np.zeros((number_of_grids,number_of_ppmax))
for index in range(pcp_len):
    j = int(pcp.iloc[index]['grid number'])
    for k in ppmax:
        if pcp_bi[j,k] == 0:
            pcp_bi[j,k] = 1
            break
            
# Collect latitude of each potential point            
pcp_lati = np.zeros((number_of_grids,number_of_ppmax))
for index in range(pcp_len):
    j = int(pcp.iloc[index]['grid number'])
    for g in ppmax:
        if pcp_lati[j,g] == 0:
            pcp_lati[j,g] = pcp.iloc[index]['Latitude']
            break
            
# Collect longitude of each potential point 
pcp_longi = np.zeros((number_of_grids,number_of_ppmax))
for index in range(pcp_len):
    j = int(pcp.iloc[index]['grid number'])
    for g in ppmax:
        if pcp_longi[j,g] == 0:
            pcp_longi[j,g] = pcp.iloc[index]['Longitude']
            break
    

In [9]:
# Find out the grids that have demands in the four years but don't have potential points and 
# the existing charging points cannot meet the demand
demand_tot = np.sum(demand, axis = 1)
demand_index = np.where(demand_tot != 0)
demand_index = np.asarray(demand_index)
index = np.isin(demand_index, pcp['grid number'].unique())
no_pcp = demand_index[np.where(index == False)]


In [10]:
# Initialization
satisfication_max = np.array([3500, 5200, 50500])
charger_cost = np.array([500, 4500, 33000])
station_cost = 1200
consumption_rate = 0.18 # kWh/km
price = np.array([0.20, 0.25, 0.30]) # pounds per kwh
average_price = 0.25 # pounds per kwh

In [11]:
# Create the variables
point_used = np.array([xp.var( name='point_used_{0}_{1}_{2}'.format(j+1,k+1,t+1), vartype = xp.binary)
                    for j in grids for k in ppmax for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_ppmax,number_of_years)
charger = np.array([xp.var( name='charger_{0}_{1}_{2}_{3}'.format(i+1,j+1,k+1,t+1), vartype = xp.integer)
                    for i in types for j in grids for k in ppmax for t in years], dtype=xp.npvar).reshape(number_of_types,number_of_grids,number_of_ppmax,number_of_years)
rapid_bi = np.array([xp.var( name='rapid_bi_{0}_{1}_{2}'.format(j+1,k+1,t+1))
                    for j in grids for k in ppmax for t in years], dtype=xp.npvar).reshape(number_of_grids,number_of_ppmax,number_of_years)

prob.addVariable(point_used, charger, rapid_bi)

In [12]:
# Only when the charging point is used, there exists chargers
prob.addConstraint(xp.Sum(charger[i,j,k,t] for i in types) <= 100000*point_used[j,k,t] 
                   for j in grids for k in ppmax for t in years)

# Only potential charging point can be used
prob.addConstraint(point_used[j,k,t] <= pcp_bi[j,k] 
                   for j in grids for k in ppmax for t in years)
 
# A station cannot only include rapid chargers
prob.addConstraint(charger[2,j,k,t] <= 100000*(1-rapid_bi[j,k,t]) 
                   for j in grids for k in ppmax for t in years)
prob.addConstraint(1-(charger[0,j,k,t]+charger[1,j,k,t]) <= 100000*rapid_bi[j,k,t] 
                   for j in grids for k in ppmax for t in years)

# Meet the demand requirements
prob.addConstraint(xp.Sum(satisfication_max[i]*(xp.Sum(charger[i,j,k,t] for k in ppmax) 
                          + xp.Sum(charger_0[i,j,p] for p in cpmax)) for i in types) >= demand[j,t] 
                   for j in grids for t in years if j not in no_pcp)

# The charging stations which are going to be used won't be removed for the next four years
prob.addConstraint(point_used[j,k,t] <= point_used[j,k,t+1] 
                   for j in grids for k in ppmax for t in range(number_of_years-1))

# The charging infrastructures that have already been established won't be removed for the next four years
prob.addConstraint(charger[i,j,k,t] <= charger[i,j,k,t+1] 
                   for i in types for j in grids for k in ppmax for t in range(number_of_years-1))

# The amount of chargers should be positive
prob.addConstraint(charger[i,j,k,t] >= 0 
                   for i in types for j in grids for k in ppmax for t in years)

In [13]:
def to_distance(lati1, longi1, lati2, longi2):
    '''
    Calculate the distance between two points with latitude and longitude 
    (lati1, longi1) and (lati2, longi2).
    '''
    lati1 = lati1*np.pi/180
    longi1 = longi1*np.pi/180
    lati2 = lati2*np.pi/180
    longi2 = longi2*np.pi/180
    dist = 6371*np.arccos(np.cos(lati1)*np.cos(lati2)*np.cos(longi1-longi2) + np.sin(lati1)*np.sin(lati2))
    return dist

# Calculate the average distance from each charging point to all interest points in the same grid
dist = np.zeros((number_of_grids,number_of_ppmax))
for j in grids:
    poi_j = np.sum(poi_lati[j,:] > 0)
    for k in ppmax:
        if pcp_lati[j,k] == 0:
            break
        dist_k = 0
        for g in poimax:
            if poi_lati[j,g] == 0:
                break
            dist_k += to_distance(pcp_lati[j,k], pcp_longi[j,k], poi_lati[j,g], poi_longi[j,g])
        if poi_j != 0:
            dist[j,k] = dist_k/poi_j 


In [14]:
prob.setObjective(average_price*xp.Sum(- demand[j,t] + xp.Sum(satisfication_max[i]*(xp.Sum(charger[i,j,k,t] for k in ppmax) + xp.Sum(charger_0[i,j,p] for p in cpmax)) for i in types) for j in grids for t in years) +
                  xp.Sum(charger_cost[i]* charger[i,j,k,3] for j in grids for k in ppmax for i in types) +
                  xp.Sum(station_cost * point_used[j,k,3] for j in grids for k in ppmax) +
                  consumption_rate*average_price*xp.Sum(point_used[j,k,t] * dist[j,k] for j in grids for k in ppmax for t in years), 
                  sense = xp.minimize)


In [15]:
prob.write("problem","lp")

In [16]:
prob.solve()

FICO Xpress v8.13.7, Hyper, solve started 12:46:59, Nov 28, 2022
Heap usage: 126MB (peak 126MB, 74MB system)
Minimizing MILP EV charging station using up to 8 threads, with these control settings:
OUTPUTLOG = 1
Original problem has:
    314900 rows       156600 cols       686880 elements    125280 globals
Presolved problem has:
      2629 rows         2737 cols         8358 elements      2737 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 138MB (peak 302MB, 74MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+05] / [ 6.25e-02,  1.99e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  2.79e+05] / [ 1.00e+00,  1.20e+01]
  Objective      [min,max] : [ 1.03e-03,  4.56e+04] / [ 1.58e-03,  4.56e+04]
Autoscaling applied standard scaling

Symmetric problem: generators: 72, support set: 846
 Number of orbits: 170, largest orbit: 13
 Row orbits: 157, row support: 756
Will try to keep bran

In [17]:
x = prob.getSolution(charger)
y = prob.getSolution(point_used)
x_1 = np.sum(x[:,:,:,0], axis = 0)
x_2 = np.sum(x[:,:,:,1], axis = 0)
x_3 = np.sum(x[:,:,:,2], axis = 0)
x_4 = np.sum(x[:,:,:,3], axis = 0)

station_num_1 = np.sum(x_1 > 0, axis = 1)
station_num_2 = np.sum(x_2 > 0, axis = 1)
station_num_3 = np.sum(x_3 > 0, axis = 1)
station_num_4 = np.sum(x_4 > 0, axis = 1)

In [19]:
dist_center = demand_data['Distance from Centre'].to_numpy()
dist_center = np.insert(dist_center, 0, [0], axis=0)

df1 = pd.DataFrame()
df1['Grid'] = range(1, number_of_grids)
df1['Distance from Centre'] = demand_data['Distance from Centre']
df1['Number of chargers'] = np.sum(x_1, axis = 1)[1:436]
df1['Number of stations'] = station_num_1[1:436]
df1 = df1.drop(df1[df1['Number of chargers'] == 0].index)
df1 = df1.sort_values(['Distance from Centre'])
display(df1)


,Grid,Distance from Centre,Number of chargers,Number of stations
228,229,604.578858,1.0,1
227,228,606.913349,1.0,1
159,160,2332.555878,1.0,1
145,146,2849.811425,1.0,1
146,147,3050.880206,1.0,1
133,134,3790.012694,5.0,1
368,369,5999.689193,5.0,1


In [20]:
df2 = pd.DataFrame()
df2['Grid'] = range(1, number_of_grids)
df2['Distance from Centre'] = demand_data['Distance from Centre']
df2['Number of chargers'] = np.sum(x_2, axis = 1)[1:436]
df2['Number of stations'] = station_num_2[1:436]
df2 = df2.drop(df2[df2['Number of chargers'] == 0].index)
df2 = df2.sort_values(['Distance from Centre'])
display(df2)

,Grid,Distance from Centre,Number of chargers,Number of stations
214,215,270.076114,2.0,1
213,214,275.262283,2.0,1
228,229,604.578858,2.0,1
227,228,606.913349,1.0,1
200,201,612.375019,2.0,1
199,200,614.679901,2.0,1
212,213,820.508431,3.0,1
229,230,978.429315,1.0,1
241,242,1120.515437,1.0,1
186,187,1127.697550,1.0,1


In [21]:

df3 = pd.DataFrame()
df3['Grid'] = range(1, number_of_grids)
df3['Distance from Centre'] = demand_data['Distance from Centre']
df3['Number of chargers'] = np.sum(x_3, axis = 1)[1:436]
df3['Number of stations'] = station_num_3[1:436]
df3 = df3.drop(df3[df3['Number of chargers'] == 0].index)
df3 = df3.sort_values(['Distance from Centre'])
display(df3)


,Grid,Distance from Centre,Number of chargers,Number of stations
214,215,270.076114,4.0,1
213,214,275.262283,6.0,1
228,229,604.578858,3.0,1
227,228,606.913349,3.0,1
200,201,612.375019,4.0,1
199,200,614.679901,6.0,1
212,213,820.508431,7.0,1
229,230,978.429315,1.0,1
201,202,983.265728,1.0,1
241,242,1120.515437,1.0,1


In [22]:
df4 = pd.DataFrame()
df4['Grid'] = range(1, number_of_grids)
df4['Distance from Centre'] = demand_data['Distance from Centre']
df4['Number of chargers'] = np.sum(x_4, axis = 1)[1:436]
df4['Number of stations'] = station_num_4[1:436]
df4 = df4.drop(df4[df4['Number of chargers'] == 0].index)
df4 = df4.sort_values(['Distance from Centre'])
display(df4)


,Grid,Distance from Centre,Number of chargers,Number of stations
214,215,270.076114,8.0,1
213,214,275.262283,12.0,1
228,229,604.578858,5.0,1
227,228,606.913349,5.0,1
200,201,612.375019,7.0,1
199,200,614.679901,12.0,1
212,213,820.508431,12.0,1
229,230,978.429315,1.0,1
201,202,983.265728,3.0,1
241,242,1120.515437,2.0,1


In [23]:
charger_tot_last = 0
station_tot_last = 0
for t in years:
    print(f'In the year {t+1}:\n')
    charger_tot_current = np.sum(np.sum(np.sum(x[:,:,:,t], axis = 0), axis = 1))
    print(f'set up {int(charger_tot_current)} chargers in total.\n')
    station_tot_current = np.sum(np.sum(y[:,:,t], axis = 1))
    print(f'construct {int(station_tot_current)} stations in total.\n')
    
print(f'The objective function value is {prob.getObjVal()}')

In the year 1:

set up 15 chargers in total.

construct 35 stations in total.

In the year 2:

set up 84 chargers in total.

construct 54 stations in total.

In the year 3:

set up 160 chargers in total.

construct 75 stations in total.

In the year 4:

set up 241 chargers in total.

construct 81 stations in total.

The objective function value is 913870.4520291267
